# SparkML with Dataproc Serverless

## Overview

This notebook tutorial demonstrates the execution of Apache SparkML jobs using Dataproc Serverless. The example focuses on a typical machine learning pipeline, covering stages such as data ingestion and cleaning, feature engineering, modeling, and model evaluation.

### Objective

This tutorial guides you through the execution of an Apache SparkML job that involves fetching data from the lakehouse, conducting Exploratory Data Analysis (EDA), data cleaning, feature engineering, model training, model evaluation, result output, and saving the model to a Cloud Storage bucket.

The tutorial leverages the following Google Cloud ML services:
- `Dataproc`
- `BigQuery`
- `Vertex AI Training`

Key steps include:

- Setting up a Google Cloud project and Dataproc serverless.
- Configuring the spark-bigquery-connector.
- Ingesting data from the lakehouse into a Spark DataFrame.
- Conducting Exploratory Data Analysis (EDA).
- Visualizing data samples.
- Cleaning the data.
- Selecting features.
- Training the model.
- Outputting results.
- Saving the model to a Cloud Storage bucket.

### Dataset

The [NYC TLC (Taxi and Limousine Commission) Trips](https://console.cloud.google.com/marketplace/product/city-of-new-york/nyc-tlc-trips) (New York taxi and limosine trips data) dataset is available in [BigQuery Public Datasets](https://cloud.google.com/bigquery/public-data).

## Tutorial

### Set your project ID

In [ ]:
# Retrieve the current active project and store it as a list of strings.
PROJECT_ID = !gcloud config get-value project

# Extract the project ID from the list.
PROJECT_ID = PROJECT_ID[0] if PROJECT_ID else None

### Get a Cloud Storage bucket URI

In [ ]:
# Define the prefix of the bucket created via Terraform.
BUCKET_PREFIX = "gcp-lakehouse-model"

# Retrieve the Cloud Storage bucket URI for storing the machine learning model.
BUCKET_URI = !gcloud storage buckets list --format='value(name)' --filter='name:{BUCKET_PREFIX}*'

# Extract the bucket URI from the list.
BUCKET_URI = BUCKET_URI[0] if BUCKET_URI else None

### Import required libraries

In [ ]:
import os

import matplotlib.pyplot as plt
import seaborn as sns
from geopandas import gpd
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import GBTRegressor
# A Spark Session is how you interact with Spark SQL to create Dataframes
from pyspark.sql import SparkSession
# PySpark functions
from pyspark.sql.functions import col, floor, unix_timestamp

### Initialize the SparkSession

When setting up your SparkSession to work with Apache Spark and BigQuery, ensure that you include the [spark-bigquery-connector](https://github.com/GoogleCloudDataproc/spark-bigquery-connector).

In [ ]:
VER = "0.34.0"
FILE_NAME = f"spark-bigquery-with-dependencies_2.12-{VER}.jar"
connector = f"gs://spark-lib/bigquery/{FILE_NAME}"

# Set the log level to ERROR for the SparkSession to suppress the warning

# Initialize the SparkSession.
spark = (
    SparkSession.builder.appName("spark-ml-taxi")
    .config("spark.jars", connector)
    .config("spark.logConf", "false")
    .getOrCreate()
)

### Fetch data

In this tutorial, you will be working with the NYC Taxi dataset, focusing on trips that occurred in 2022. Given the original dataset's substantial size of 36 million rows, completing this tutorial with the entire dataset would demand considerable time, resources, and associated costs. To address this, you will be working with a representative 5% sample of the data, allowing us to achieve the tutorial objectives more efficiently.

Please note that utilizing a smaller sample size helps minimize the computational resources required and is a common practice for tutorial purposes. The concepts and techniques demonstrated with the sampled data remain applicable to the full dataset.

In [ ]:
# Load NYC_taxi in Github Activity Public Dataset from BigQuery.
taxi_df = (
    spark.read.format("bigquery")
    .option("table", f"{PROJECT_ID}.gcp_primary_staging.new_york_taxi_trips_tlc_yellow_trips_2022")
    .load()
)

# Sample data to minimize the cost. 
# You can try different values in the parameter fraction, or remove the next line to use all.
taxi_df = taxi_df.sample(fraction=0.05, seed=42)

### Perform Exploratory Data Analysis (EDA)

Initiate the Exploratory Data Analysis (EDA) phase to uncover patterns, relationships, and anomalies within the dataset. Employ various graphical and visualization techniques to gain insights into the data's characteristics.

In [ ]:
taxi_df.printSchema()

Remove unnecessary columns and assess the null counts of the fields. Keep in mind that `pickup_location_id` and `dropoff_location_id` columns have been converted to strings but should be integers. To use these columns effectively, reformat these values.

In [ ]:
# Choose categorical columns to drop.
columns_to_drop = [
    'vendor_id',
    'rate_code',
    'store_and_fwd_flag',
    'payment_type',
    'data_file_year',
    'data_file_month'
]

# Drop the specified columns and convert pickup_location_id and dropoff_location_id to integers.
taxi_df = (
    taxi_df
    .drop(*columns_to_drop)
    .withColumn("start_zone_id", col("pickup_location_id").cast("int"))
    .withColumn("end_zone_id", col("dropoff_location_id").cast("int"))
)

# Display summary statistics and preview the modified DataFrame.
taxi_df.describe().show()
taxi_df.show(10)

From this summary, several key insights emerge:
  * There are over 1 million trip histories for Yellow Taxi in 2022, which represents approximately 5% of the total trips.
  * Certain trip histories contain abnormalies, such as very large numbers in `trip_distance`.
  * The dataset's entries had included precise latitude and longitude data for both pickup and dropoff locations up until the year 2016. However, due to [privacy concerns](https://agkn.wordpress.com/2014/09/15/riding-with-the-stars-passenger-privacy-in-the-nyc-taxicab-dataset/), this granular location information was discontinued. Instead, the pickup_location_id and dropoff_location_id now align with the [NYC Taxi Zones](https://data.cityofnewyork.us/Transportation/NYC-Taxi-Zones/d3c5-ddgc), offering approximations based on the NYC Department of City Planning’s Neighborhood Tabulation Areas (NTAs) to indicate approximate neighborhood locations.

Based on the information above, the following filtering criteria can be used to remove unrealistic values from the dataset.
  * Exclude entries with excessively large `trip_distance` values, surpassing 10,000 miles.
  * Exclude entries with null and negative values.

In [ ]:
taxi_df = taxi_df.where(
    (col("trip_distance") < 10000) &
    (col("fare_amount") > 0) &
    (col("extra") >= 0) &
    (col("mta_tax") >= 0) &
    (col("tip_amount") >= 0) &
    (col("tolls_amount") >= 0) &
    (col("imp_surcharge") >= 0) &
    (col("airport_fee") >= 0) &
    (col("total_amount") > 0)
).dropna()

Use the `unix_timestamp()` function to trasform `pickup_datetime` and `dropoff_datetime` into Unix Timestamp type.

Once you have converted `pickup_datetime` and `dropoff_datetime` to `start_time` and `end_time` as Unix Timestamps, you can manipulate these timestamps to generate more compelling columnnar data.

In [ ]:
# Convert the datetime from string to Unix timestamp.
taxi_df = taxi_df.withColumn("start_time", unix_timestamp(col("pickup_datetime")))
taxi_df = taxi_df.withColumn("end_time", unix_timestamp(col("dropoff_datetime")))

# Determine if it's a weekday.
taxi_df = taxi_df.withColumn(
    "is_weekday",
    ((floor(col("start_time") / 86400) + 4) % 7 > 0)
    & ((floor(col("start_time") / 86400) + 4) % 7 < 6),
)

# Convert start_time to start_time_in_hour.
taxi_df = taxi_df.withColumn(
    "start_time_in_hour", floor((col("start_time") % 86400) / 60 / 60)
)

# Calculate trip_duration.
taxi_df = taxi_df.withColumn("trip_duration", col("end_time") - col("start_time"))

# Drop unnecessary columns.
taxi_df = taxi_df.drop("pickup_datetime", "dropoff_datetime")

### Perform Feature Engineering

While the Taxi dataset contains trips for all NYC boroughs, the precise location information is categorized using `NYC Taxi zones`. To make the location ID with significance, acquire the GeoJSON format of NYC Taxi zones from the BigQuery public dataset.

In [ ]:
# Load the GeoJSON format of NYC Taxi zones from the BigQuery public dataset.
geo_df = (
    spark.read.format("bigquery")
    .option("table", "bigquery-public-data.new_york_taxi_trips.taxi_zone_geom")
    .load()
)

# Convert Spark DataFrame into Pandas DataFrame to integrate with the GeoPandas library.
geo_pd = geo_df.toPandas()

# Create a GeoDataFrame based on the central point of each taxi zone, separated by latitude and longitude.
geo_pd['long'] = gpd.GeoSeries.from_wkt(geo_pd['zone_geom']).centroid.x
geo_pd['lat'] = gpd.GeoSeries.from_wkt(geo_pd['zone_geom']).centroid.y

# Drop unnecessary columns.
geo_pd = geo_pd[["zone_id", "long", "lat"]]

# Convert back to a Spark DataFrame.
geo_spark_df = spark.createDataFrame(geo_pd)

# Join taxi_df with geographic position for each start_zone_id.
taxi_zone_df = (taxi_df
        .join(geo_spark_df, taxi_df.start_zone_id == geo_spark_df.zone_id)
        .withColumnRenamed("long", "start_long")
        .withColumnRenamed("lat", "start_lat")
        .drop("zone_id")
        .join(geo_spark_df, taxi_df.end_zone_id == geo_spark_df.zone_id)
        .withColumnRenamed("long", "end_long")
        .withColumnRenamed("lat", "end_lat")
        .drop("zone_id")
       )

taxi_zone_df.printSchema()
taxi_zone_df.describe().show()

Same as `trip_distance`, `trip_duration` also has some extreme values. Remove unrealistic values from the dataset.

In [ ]:
# Filter trips occurring between same taxi zones and exceeding where trip_duration is more than 28800 seconds (8 hours).
taxi_df = taxi_zone_df.where(
    (col("trip_duration") < 28800) &
    (col("start_zone_id") != col("end_zone_id"))
)

# Drop unnecessary columns
taxi_df = taxi_df.drop("pickup_location_id", "dropoff_location_id")

Examine the distributions for the numerical columns.

In [ ]:
# Define columns to be converted to numerical type.
columns_to_convert = [
    "trip_distance",
    "fare_amount",
    "extra",
    "mta_tax",
    "tip_amount",
    "tolls_amount",
    "imp_surcharge",
    "airport_fee",
    "total_amount"
]

# Convert Spark DataFrame into a Pandas DataFrame.
taxi_pd = taxi_df.toPandas()

# Convert columns of "object" type to the float type.
taxi_pd[columns_to_convert] = taxi_pd[columns_to_convert].astype(float)

# Display information about the Pandas DataFrame.
taxi_pd.info()

Filter out outliers and organize the data into a box plot and histogram.

In [ ]:
# Define the columns of interest for visualization.
TAXI_COLUMNS_TO_SHOW = {
    "trip_distance",
    "trip_duration",
    "total_amount",
    "fare_amount",
    'extra',
    'mta_tax',
    'tip_amount',
    'tolls_amount',
    'imp_surcharge',
    'airport_fee',
}

# Filter the DataFrame to include data within reasonable ranges.
taxi_pd_filtered = taxi_pd.query(
    "trip_distance > 0 and trip_distance < 20 \
    and trip_duration > 0 and trip_duration < 10000"
)

# Scatter plot to visualize the relationship between trip_distance and trip_duration.
sns.relplot(
    data=taxi_pd_filtered,
    x="trip_distance",
    y="trip_duration",
    kind="scatter",
)

# Disable the warning for too many open figures
plt.rcParams['figure.max_open_warning'] = 0

# Box plots and histograms for the specified columns.
for column in taxi_pd_filtered.columns:
    if column in TAXI_COLUMNS_TO_SHOW:
        _, ax = plt.subplots(1, 2, figsize=(10, 4))
        taxi_pd_filtered[column].plot(kind="box", ax=ax[0])
        taxi_pd_filtered[column].plot(kind="hist", ax=ax[1])
        plt.title(column)
        plt.figure()
plt.show()

The data is right-skewed, and a positive correlation is observed between `trip_distance` and `trip_duration`. The majority of trips are completed within an hour (3600 sec).

### Feature Selection

Not all features in our dataset will be utilized for training. After selecting the following columns as features, they must be assembled using `VectorAssembler()`, a feature transformer that consolidates multiple columns into a vector column.

In [ ]:
# List of selected features for training the model.
feature_cols = [
    "passenger_count",
    "is_weekday",
    "start_time_in_hour",
    "trip_distance",
    "start_time",
    "end_time",
    "is_weekday",
    "start_time_in_hour",
    "start_long",
    "start_lat",
    "end_long",
    "end_lat",
    "total_amount",
    "fare_amount",
    'extra',
    'mta_tax',
    'tip_amount',
    'tolls_amount',
    'imp_surcharge',
    'airport_fee',
]

# Create a VectorAssembler with specified input and output columns.
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")

# Transform each column into vector form using the VectorAssembler.
taxi_transformed_data = assembler.transform(taxi_df)

# Randomly split the transformed data into training and test sets.
(taxi_training_data, taxi_test_data) = taxi_transformed_data.randomSplit([0.95, 0.05])

### Training the Model

PySpark offers various Regression Models, including  `LinearRegression`, `GeneralizedLinearRegression`, `DecisionTreeRegressor`, `RandomForestRegressor`, and `GBTRegressor`.

`GBTRegressor`, known as `Gradient Boosted Trees` is a widely-used regression method employing ensembles of decision trees. This algorithm iteratively trains decision trees to minimize a loss function.
Although this model is computationally expensive, empirical testing has demonstrated that `GBT Regressor` yields the most favorable results.

In [ ]:
# Define GBTRegressor model with specified input, output, and prediction columns.
gbt = GBTRegressor(
    featuresCol="features",
    labelCol="trip_duration",
    predictionCol="pred_trip_duration",
)

# Define an evaluator for calculating the R2 score.
evaluator_r2 = RegressionEvaluator(
    labelCol=gbt.getLabelCol(), predictionCol=gbt.getPredictionCol(), metricName="r2"
)

# Define an evaluator for calculating the RMSE error.
evaluator_rmse = RegressionEvaluator(
    labelCol=gbt.getLabelCol(), predictionCol=gbt.getPredictionCol(), metricName="rmse"
)

In [ ]:
# Train a Gradient Boosted Trees (GBT) model on the Taxi dataset. This process may take several minutes.
taxi_gbt_model = gbt.fit(taxi_training_data)

# Get predictions for the Taxi dataset using the trained GBT model.
taxi_gbt_predictions = taxi_gbt_model.transform(taxi_test_data)

In [ ]:
# Evaluate the R2 score for the Taxi dataset predictions.
taxi_gbt_accuracy_r2 = evaluator_r2.evaluate(taxi_gbt_predictions)
print(f"Taxi Test GBT R2 Accuracy = {taxi_gbt_accuracy_r2}")

# Evaluate the Root Mean Squared Error (RMSE) for the Taxi dataset predictions.
taxi_gbt_accuracy_rmse = evaluator_rmse.evaluate(taxi_gbt_predictions)
print(f"Taxi Test GBT RMSE Accuracy = {taxi_gbt_accuracy_rmse}")

### View the result

The trained model for the Taxi dataset yields an R2 score ranging betweein approximately 83-87% and a Root Mean Square Error(RMSE) of 200-300, the exact values dependent on the chose samples. In practical machine learning projects, cross-validation emerges as a crucial tool, enhancing data utilization. However, due to its iterative nature, cross-validation demands a substantial investment of time and computation resources to complete. For additional insights, refer to [Cross-validation (statistics)](https://en.wikipedia.org/wiki/Cross-validation_%28statistics%29).

### Save the model to a Cloud Storage path

To ensure the preservation and accessibility of the trained model, it can be saved to a Cloud Storage path.

In [ ]:
# Save the trained model to a Cloud Storage path
taxi_gbt_model.write().overwrite().save(f"gs://{BUCKET_URI}/")